In [0]:
from google.colab import files

In [363]:
englishtxt = files.upload()
spanishtxt = files.upload()

Saving EnglishNew.txt to EnglishNew (7).txt


Saving SpanishNew.txt to SpanishNew (7).txt


In [0]:
# spanishtest = files.upload()

In [0]:
# temp_esp_txt = [ele.lower() for ele in spanishtest["SpanishTest.txt"].decode("utf-8", "ignore").replace("\r\n",".").split('.')]
# esp_txt = []
# for ele in temp_esp_txt:
#   if len(ele)>3:
#     if ele[0]==" ":
#       esp_txt.append(ele[1:])  
#       continue
#     esp_txt.append(ele)

In [0]:
import collections

import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [367]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7535592162166123059
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7402421152871166542
physical_device_desc: "device: XLA_CPU device"
]


In [368]:
english_sentences = [ele.lower() for ele in englishtxt["EnglishNew.txt"].decode("utf-8", "ignore").split('\r\n')]

len(english_sentences)

116

In [369]:
spanish_sentences = [ele.lower() for ele in spanishtxt["SpanishNew.txt"].decode("utf-8", "ignore").split('\r\n')]
len(spanish_sentences)

116

In [370]:
count=0
for sample_i in range(5):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_esp Line {}:  {}\n'.format(sample_i + 1, spanish_sentences[sample_i]))

small_vocab_en Line 1:  cystadenocarcinoma is a rare adenomatous pancreatic cancer that comes from the malignant degeneration of a mucous cystadenoma and is manifested by high abdominal pain and a palpable abdominal mass.
small_vocab_esp Line 1:  el cistoadenocarcinoma es un cncer pancretico adenomatoso raro que proviene de la degeneracin maligna de un cistoadenoma mucoso y se manifiesta por dolor abdominal alto y una masa abdominal palpable.

small_vocab_en Line 2:  the diagnosis of cystadenocarcinoma is made by ct or mri of the abdomen, which usually shows a cystic mass that contains remains; the mass can be misunderstood as a necrotic adenocarcinoma or a pancreatic pseudocyst.
small_vocab_esp Line 2:  el diagnstico del cistadenocarcinoma se realiza por tc o rm de abdomen, que suele mostrar una masa qustica que contiene restos; la masa puede malinterpretarse como un adenocarcinoma necrtico o un seudoquiste pancretico.

small_vocab_en Line 3:  unlike ductal adenocarcinoma, cystadenoca

In [371]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
spanish_words_counter = collections.Counter([word for sentence in spanish_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Spanish words.'.format(len([word for sentence in spanish_sentences for word in sentence.split()])))
print('{} unique Spanish words.'.format(len(spanish_words_counter)))
print('10 Most common words in the Spanish dataset:')
print('"' + '" "'.join(list(zip(*spanish_words_counter.most_common(10)))[0]) + '"')

2185 English words.
924 unique English words.
10 Most common words in the English dataset:
"the" "of" "and" "is" "to" "a" "or" "in" "with" "blood"

2538 Spanish words.
968 unique Spanish words.
10 Most common words in the Spanish dataset:
"de" "la" "el" "y" "los" "en" "a" "o" "una" "se"


In [0]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [0]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

In [0]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [375]:
preproc_spanish_sentences, preproc_english_sentences, spanish_tokenizer, english_tokenizer  =\
    preprocess(spanish_sentences, english_sentences)

max_spanish_sequence_length = preproc_spanish_sentences.shape[1]
max_english_sequence_length = preproc_english_sentences.shape[1]

spanish_vocab_size = len(spanish_tokenizer.word_index)
english_vocab_size = len(english_tokenizer.word_index)

print('Data Preprocessed')
print("Max Spaninsh sentence length:", max_spanish_sequence_length)
print("Max English sentence length:", max_english_sequence_length)
print("Spaninsh vocabulary size:", spanish_vocab_size)
print("English vocabulary size:", english_vocab_size)

Data Preprocessed
Max Spaninsh sentence length: 53
Max English sentence length: 46
Spaninsh vocabulary size: 841
English vocabulary size: 783


In [376]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [0]:
def bd_model(input_shape, output_sequence_length, spanish_vocab_size, english_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.1))
    model.add(TimeDistributed(Dense(english_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [0]:
# # : Reshape the input
# tmp_x = pad(preproc_spanish_sentences, preproc_english_sentences.shape[1])
# tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2], 1))
# esp_test = pad(preproc_spanish_sentences[:11], preproc_english_sentences[:11].shape[1])
# esp_test = esp_test.reshape((-1, preproc_english_sentences[:11].shape[-2], 1))

# # TODO: Train the neural network
# bd_rnn_model = bd_model(
#     tmp_x.shape,
#     preproc_english_sentences[11:].shape[1],
#     len(spanish_tokenizer.word_index)+1,
#     len(english_tokenizer.word_index)+1)

# bd_rnn_model.summary()

# bd_rnn_model.fit(tmp_x, preproc_english_sentences, batch_size=7, epochs=30, validation_split=0.05)

In [379]:
# TODO: Reshape the input
tmp_x = pad(preproc_spanish_sentences[11:], preproc_english_sentences[11:].shape[1])
tmp_x = tmp_x.reshape((-1, preproc_english_sentences[11:].shape[-2], 1))
esp_test = pad(preproc_spanish_sentences[:11], preproc_english_sentences[:11].shape[1])
esp_test = esp_test.reshape((-1, preproc_english_sentences[:11].shape[-2], 1))

# TODO: Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_english_sentences[11:].shape[1],
    len(spanish_tokenizer.word_index)+1,
    len(english_tokenizer.word_index)+1)

bd_rnn_model.summary()

bd_rnn_model.fit(tmp_x, preproc_english_sentences[11:], batch_size=7, epochs=30, validation_split=0.05)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_22 (Bidirectio (None, 46, 256)           99840     
_________________________________________________________________
time_distributed_43 (TimeDis (None, 46, 512)           131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 46, 512)           0         
_________________________________________________________________
time_distributed_44 (TimeDis (None, 46, 784)           402192    
Total params: 633,616
Trainable params: 633,616
Non-trainable params: 0
_________________________________________________________________
Train on 99 samples, validate on 6 samples
Epoch 1/30
99/99 [==============================] - 7s 72ms/step - loss: 4.7371 - acc: 0.5556 - val_loss: 2.9859 - val_acc: 0.5761
Epoch 2/30
99/99 [==============================] - 1s 14ms/step - loss

In [384]:
esp_txt_len = esp_test.shape[0] 
for i_esp_txt in range(esp_txt_len):
  # Print prediction(s)
#   print("\nPrediction:")
  print(logits_to_text(bd_rnn_model.predict(esp_test[i_esp_txt:i_esp_txt+1])[0], english_tokenizer))
#   print("\nCorrect Translation:")
#   print(english_sentences[i_esp_txt:i_esp_txt+1])

#   print("\nOriginal text:")
#   print(spanish_sentences[i_esp_txt:i_esp_txt+1])

if cancer be of given prompt of any it or termination 2 to 1 to output measured after fetus 4 once once in <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
other gestational patients no an to have bile to specific or pruritus to be disease years to 1 and to obstructive 1 accessible ee or in 20 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
transfusions involve should delivery delivery as be <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
the on given given within maternal measurement high about and to the to bile bile episode termination to failure indicating measurement of heart an to output failure heart to orally <PAD> <PAD> <PAD> <PAD> <PAD> <PAD